In [1]:

# coding: utf-8

import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import psycopg2 as psy
import os
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import r2_score
from matplotlib import *
import sys
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)
from sklearn import metrics
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.cross_validation import StratifiedKFold, permutation_test_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
import matplotlib.patheffects as path_effects
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# some helper functions

def pop_year_clean(year):
    year = str(year)
    year = year.replace(" ", "")
    try:
        year = int(year)
    except:
        year = year
    return year


def state_fix(code):
    code = str(code)
    if len(code) == 1:
        code = "0" + code
    return code

def county_fix(code):
    code = str(code)
    if len(code) == 1:
        code = "00" + code
    elif len(code) ==2:
        code = "0" + code
    return code

def col_includer(year, col):
    if year in col:
        return col
    else:
        return 0

#used for growth data
def growth_convert(df, year):
    final_cols = []
    df_cols = df.columns
    last_year = str(year - 1)
    year = str(year)
    df_cols_this = df_cols.map(lambda x: col_includer(year, x))
    df_cols_last = df_cols.map(lambda x: col_includer(last_year, x))

    #cleaning this year col
    df_cols_this = pd.DataFrame(df_cols_this)
    df_cols_this = df_cols_this[df_cols_this[0]!=0]
    final_cols.append(df_cols_this.iloc[0,0])
    #pulling last year's columns
    df_cols_last = pd.DataFrame(df_cols_last)
    df_cols_last = df_cols_last[df_cols_last[0]!=0]
    final_cols.append(df_cols_last.iloc[1,0])
    final_cols.append(df_cols_last.iloc[12,0])
    final_cols.append(df_cols_last.iloc[13,0])
    final_cols.append(df_cols_last.iloc[14,0])
    final_cols.append('STATE')
    final_cols.append('COUNTY')
    df = df[final_cols]
    df['STATE'] = df['STATE'].apply(lambda x: state_fix(x))
    df['COUNTY'] = df['COUNTY'].apply(lambda x: county_fix(x))
    df['string_county_code'] = df['STATE'] + df['COUNTY']
    df['county_code'] = df['string_county_code'].apply(lambda x: int(x))
    df['year'] = int(year)
    df.columns = ['population_est', 'net_pop_change_raw','natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate', 'state_num', 'county_num', 'string_county_code', 'county_code', 'year']
    return df

def yearify(year):
    year = float(year)
    try:
        return int(year)
    except:
        return year

def interator(example):
    try:
        example = str(example).split('.')[0]
        return int(example)
    except:
        return example

def death_clean(death):
    try:
        return float(death)
    except:
        return np.nan

def state_extract(county):
    county = str(county)
    try:
        county = county.split(",")[1]
        county = county.replace(" ", "")
        return county
    except:
        if len(county) == 2:
            return county
        else:
            return "ERROR"


def floater(df):
    for col in df:
        try:
            col = col.apply(lambda x: float(x))
        except:
            continue
    return df

def dict_applier(df):
    dict_ex = defaultdict(lambda: 0)
    for i in range(len(df)):
        dict_ex[df['county_code'][i]] +=1
        df['count_county_code'][i] = dict_ex[df['county_code'][i]]
    return df


def plot_confusion_matrix(cm, title='Confusion matrix', labels=['Decreasing','Increasing'], cmap=plt.cm.Blues):

   plt.figure(figsize=(7,7))
   plt.imshow(cm, interpolation='nearest', cmap=cmap)

   tick_marks = np.arange(2)
   plt.xticks(tick_marks, labels)
   plt.yticks(tick_marks, labels)

   plt.title(title)
   plt.ylabel('True label')
   plt.xlabel('Predicted label')
   plt.colorbar()
   plt.tight_layout()

   width, height = cm.shape

   for x in xrange(width):
       for y in xrange(height):
           plt.annotate(str(cm[x][y]), xy=(y, x),
                       horizontalalignment='center',
                       verticalalignment='center',
                       color = 'white',
                       fontsize=18).set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),
                                                      path_effects.Normal()]) #The last line here adds a text outline



#reading in unemployment data

unemp_03_14 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/unemp_tot_03_14.csv')
unemp_03_14 = unemp_03_14.iloc[:,1:]

#reading in poverty data
pov_county_year_03_14 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pov_county_year_03_14.csv')
pov_county_year_03_14 = pov_county_year_03_14.iloc[:,1:]


#  removed from model, only used for name, state conversion
cdc_03_14 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/cdc_03_14.csv')
cdc_03_14 = cdc_03_14.iloc[:,2:]

name_codes = cdc_03_14.loc[:,['county_code', 'County']] #grabbing this
name_codes['county_code'] = name_codes['county_code'].apply(lambda x: int(x))
name_codes.index=name_codes['county_code']
name_codes = name_codes.iloc[:,1:]
name_codes = name_codes.to_dict()
name_codes = name_codes['County']


In [2]:
name_codes.keys()

[45063,
 45077,
 45079,
 45083,
 45091,
 41005,
 4025,
 47157,
 41017,
 47163,
 41029,
 41033,
 41035,
 41039,
 41043,
 41047,
 41051,
 39017,
 41067,
 39023,
 39025,
 46099,
 39035,
 39049,
 39057,
 39061,
 37019,
 37021,
 34035,
 39081,
 37035,
 39085,
 39093,
 39095,
 35001,
 37051,
 37057,
 17099,
 35013,
 39113,
 37067,
 37071,
 35031,
 37081,
 35039,
 37089,
 35043,
 35045,
 35049,
 39151,
 39153,
 39155,
 33013,
 33015,
 33017,
 39165,
 37119,
 37129,
 47149,
 37151,
 37159,
 37183,
 12001,
 37193,
 31055,
 51550,
 17201,
 42043,
 17197,
 29047,
 27003,
 31109,
 1089,
 29077,
 27037,
 28059,
 29095,
 25001,
 29099,
 27053,
 25009,
 54003,
 25013,
 48201,
 25017,
 25021,
 25023,
 25025,
 47179,
 23005,
 12019,
 13051,
 23019,
 27123,
 23031,
 51710,
 29183,
 27137,
 25003,
 29189,
 42071,
 25005,
 21015,
 21037,
 42077,
 51760,
 51770,
 21059,
 21067,
 21071,
 51810,
 10001,
 10003,
 53077,
 21111,
 17019,
 21117,
 10005,
 17031,
 17043,
 15003,
 8001,
 45045,
 19113,
 42101,
 17

In [3]:
name_codes[31055]

'Douglas County, NE'

In [4]:


#these files were large, had to be groupby'd in seperate scripts on EC2 before reading in
pop_03 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_03_output.csv')
pop_04 =pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_04_output.csv')
pop_05 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_05_output.csv')
pop_06 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_06_output.csv')
pop_07 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_07_output.csv')
pop_08 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_08_output.csv')
pop_09 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_09_output.csv')
pop_10_14 = pd.read_csv('/Users/HudsonCavanagh/Documents/csv_out_duplicate/pop_10_15_output.csv')

#standardizing formatting to merge
pop_10_14 = pop_10_14.loc[:,['year', 'county_code', 'pop_sub_15', 'pop_15-34', 'pop_35-54', 'pop_55+','pop_black','pop_white','pop_hisp','pop_asian', 'pop_male', 'population', 'TOT_POP']]

#merged population data
pop_03_14 = pd.concat([pop_03, pop_04, pop_05, pop_06, pop_07, pop_08, pop_09, pop_10_14], axis=0) #this works 1) need to make floats 2) need more data
pop_03_14['year'] = pop_03_14['year'].apply(lambda x: pop_year_clean(x)) #fixing integer conversion issue


### Making caluclations with pop DataFrame

pop_03_14['pop_total_age'] = pop_03_14['pop_sub_15'] + pop_03_14['pop_15-34'] + pop_03_14['pop_35-54'] + pop_03_14['pop_55+']
pop_03_14['pop_sub_15_prop'] = pop_03_14['pop_sub_15']/pop_03_14['pop_total_age']
pop_03_14['pop_15-34_prop'] = pop_03_14['pop_sub_15']/pop_03_14['pop_total_age']
pop_03_14['pop_35-54_prop'] = pop_03_14['pop_sub_15']/pop_03_14['pop_total_age']
pop_03_14['pop_55+_prop'] = pop_03_14['pop_sub_15']/pop_03_14['pop_total_age']
pop_03_14['pop_big_3_races'] = pop_03_14['pop_black'] + pop_03_14['pop_white'] + pop_03_14['pop_asian']
pop_03_14['pop_asian_prop'] = pop_03_14['pop_asian']/pop_03_14['pop_big_3_races']
pop_03_14['pop_white_prop'] = pop_03_14['pop_white']/pop_03_14['pop_big_3_races']
pop_03_14['pop_black_prop'] = pop_03_14['pop_black']/pop_03_14['pop_big_3_races']
# pop_03_14['pop_hisp_prop'] = pop_03_14['pop_hisp']/pop_03_14['pop_big_3_races'] #don't have good relative comparison
#excluded hispanic status is not mutually exclusive, no denominator to compare to &
pop_03_14['pop_male_prop'] = pop_03_14['pop_male'] / pop_03_14['pop_total_age']


# ## Merging Fentanyl and Naxolone data here
fent_13_14 = pd.read_csv('/Users/HudsonCavanagh/Dropbox/Capstone/non_cdc_data/drug_seizures_year.csv')
fent_13_14 = fent_13_14.iloc[:,1:]
fent_13_14.columns = ['year', 'national_opiate_seizures', 'national_fentanyl_seizures']

naxo = pd.read_csv('/Users/HudsonCavanagh/Dropbox/Capstone/non_cdc_data/naxo_years_clean.csv')
naxo.columns = ['year', 'state', 'naxo_crim', 'naxo_civ', 'naxo_third']

#### Merge Population, Population Growth Below

pop_grow_10_14 = pd.read_csv('/Users/HudsonCavanagh/Dropbox/Capstone/non_cdc_data/pop_dense_10_14.csv') #need to find where this file actually is
pop_grow_00_10 = pd.read_csv('/Users/HudsonCavanagh/Dropbox/Capstone/non_cdc_data/pop_grow_00_10.csv') #need to find where this file actually is


#for 2011 population growth, need to do it in 2 parts b/c different CSV layout
#first get 2011 pop estimate

pop_grow_11 = pop_grow_10_14
pop_grow_11['STATE'] = pop_grow_11['STATE'].apply(lambda x: state_fix(x))
pop_grow_11['COUNTY'] = pop_grow_11['COUNTY'].apply(lambda x: county_fix(x))
pop_grow_11['string_county_code'] = pop_grow_11['STATE'] + pop_grow_11['COUNTY']
pop_grow_11['county_code'] = pop_grow_11['string_county_code'].apply(lambda x: int(x))
pop_grow_11['year'] = 2011
pop_grow_11 = pop_grow_11[['POPESTIMATE2011', 'county_code', 'year']]
pop_grow_11.columns = ['population_est', 'county_code', 'year']

#second, get pop change in 2010
pop_grow_2010 = pop_grow_00_10
pop_grow_2010['year'] = 2011
pop_grow_2010['STATE'] = pop_grow_2010['STATE'].apply(lambda x: state_fix(x))
pop_grow_2010['COUNTY'] = pop_grow_2010['COUNTY'].apply(lambda x: county_fix(x))
pop_grow_2010['string_county_code'] = pop_grow_2010['STATE'] + pop_grow_2010['COUNTY']
pop_grow_2010['county_code'] = pop_grow_2010['string_county_code'].apply(lambda x: int(x))
pop_grow_2010 = pop_grow_2010[['year', 'county_code', 'NPOPCHG_2010','RNATURALINC2010','RINTERNATIONALMIG2010','RDOMESTICMIG2010']]
pop_grow_2010.columns = ['year', 'county_code', 'net_pop_change_raw', 'natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate']

#and merge
pop_grow_11 = pd.merge(pop_grow_11, pop_grow_2010, how='left', left_on=['year', 'county_code'], right_on=['year', 'county_code'])

#apply function to each year other than 2011 on two .csvs
growth_03 = growth_convert(pop_grow_00_10, 2003)
growth_04 = growth_convert(pop_grow_00_10, 2004)
growth_05 = growth_convert(pop_grow_00_10, 2005)
growth_06 = growth_convert(pop_grow_00_10, 2006)
growth_07 = growth_convert(pop_grow_00_10, 2007)
growth_08 = growth_convert(pop_grow_00_10, 2008)
growth_09 = growth_convert(pop_grow_00_10, 2009)
growth_10 = growth_convert(pop_grow_00_10, 2010)
# growth_11 = growth_convert(pop_grow_00_14, 2011) ## note this requires merging the two csvs because spans the difference
growth_12 = growth_convert(pop_grow_10_14, 2012)
growth_13 = growth_convert(pop_grow_10_14, 2013)
growth_14 = growth_convert(pop_grow_10_14, 2014)
#merge all population growth features - including special 2011
grow_03_14 = pd.concat([growth_03, growth_04, growth_05, growth_06, growth_07, growth_08, growth_09, growth_10, pop_grow_11, growth_12, growth_13, growth_14], axis=0)


# ## Re-merging deaths to make sure no errors above

deaths_simple = pd.read_csv('/Users/HudsonCavanagh/Dropbox/Capstone/cdc_data/deaths_simple.csv', delimiter='\t', encoding='utf-8')
#put last years alcohol
deaths_simple = deaths_simple.applymap(lambda x: str(x).replace(",", ""))
deaths_simple = deaths_simple[['County Code', 'Year', 'Deaths', 'Population', 'Crude Rate', 'Age Adjusted Rate']]
deaths_simple.columns = ['county_code', 'year', 'deaths_raw', 'cdc_population', 'death_rate_100k_cdc','death_rate_age_adjusted_cdc']

deaths_simple['year'] = deaths_simple['year'].apply(lambda x: yearify(x))
deaths_simple['county_code'] = deaths_simple['county_code'].apply(lambda x: yearify(x))
deaths_simple['deaths_raw'] = deaths_simple['deaths_raw'].apply(lambda x: death_clean(x))
deaths_simple['cdc_population'] = deaths_simple['cdc_population'].apply(lambda x: death_clean(x))
deaths_simple['death_rate_100k_cdc'] = deaths_simple['death_rate_100k_cdc'].apply(lambda x: death_clean(x))

deaths_pop = deaths_simple.groupby(by=['year', 'county_code']).sum()
deaths_pop.reset_index(inplace=1)

#merging population
deaths_pop = pd.merge(deaths_pop, pop_03_14, how='left', left_on=['year', 'county_code'], right_on=['year', 'county_code'])
# deaths_pop['population'] = deaths_pop['population_x'] #removed b/c order changed, no longer necessary

#merging poverty

deaths_pop = pd.merge(deaths_pop, pov_county_year_03_14, how='left', left_on=['year','county_code'], right_on=['year','county_code'])

#merging unemployment

deaths_pop = pd.merge(deaths_pop, unemp_03_14, how='left', left_on=['year', 'county_code'], right_on=['year', 'county_code'])

# merging fentanyl

deaths_pop = pd.merge(deaths_pop, fent_13_14, how='left', left_on=['year'], right_on=['year'])

#merging population growth

deaths_pop = pd.merge(deaths_pop, grow_03_14, how='left', left_on=['year', 'county_code'], right_on=['year', 'county_code'])
deaths_pop.dropna(axis=0, subset=['working_pop', 'employed_pop', 'deaths_raw', 'population_est'], inplace=1)

#note - need to extract state (str) to merge narcan/ naxo features (below)

###calculations that required merge
deaths_pop['drug_death_rate_100k_cdc']  = deaths_pop['deaths_raw']/(deaths_pop['cdc_population']/100000)
deaths_pop['drug_death_rate_100k']  = deaths_pop['deaths_raw']/(deaths_pop['population_est']/100000) #accepting this as target
deaths_pop['perc_pop_employed'] = deaths_pop['employed_pop']/deaths_pop['population_est']
deaths_pop['unemp_rate'] = 1 - (deaths_pop['employed_pop']/deaths_pop['working_pop']) #this is good
deaths_pop['perc_pop_working'] = deaths_pop['working_pop']/deaths_pop['population_est']
deaths_pop['pov_rate'] = deaths_pop['pov_count_tot']/ deaths_pop['population_est']
deaths_pop['pov_rate_youth'] = deaths_pop['pov_youth_count_0-17']/ deaths_pop['population_est']
deaths_pop['pop_change_rate'] = deaths_pop['net_pop_change_raw'] / deaths_pop['population_est']
deaths_pop['drug_death_rate_100k_diff_cdc_pop'] = deaths_pop['drug_death_rate_100k_cdc'] - deaths_pop['drug_death_rate_100k']
deaths_pop['constant'] = 1.0

deaths_pop.drop(1391, inplace=1)#noticed this is an outliar in plot below Coconino, AZ  pop listed as 1196.0
deaths_pop.dropna(axis=0, subset=['working_pop', 'employed_pop'], inplace=1)

##### Readding county names & states, adding state and year as dummy variables

deaths_pop[deaths_pop['county_code']==1055]


/Users/HudsonCavanagh/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/HudsonCavanagh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/HudsonCavanagh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/HudsonCavanagh/ana

,year,county_code,deaths_raw,cdc_population,death_rate_100k_cdc,AGEGRP,ORIGIN,RACE,SEX,TOT_POP,...,drug_death_rate_100k_cdc,drug_death_rate_100k,perc_pop_employed,unemp_rate,perc_pop_working,pov_rate,pov_rate_youth,pop_change_rate,drug_death_rate_100k_diff_cdc_pop,constant
1,2003.0,1055.0,11.0,103025.0,NaN,32319.0,3591.0,10773.0,3591.0,NaN,...,10.677020,10.704867,0.421879,0.065470,0.451434,0.156038,0.055695,-0.000749,-0.027847,1.0
1169,2005.0,1055.0,13.0,103174.0,NaN,32319.0,3591.0,10773.0,3591.0,NaN,...,12.600074,12.665998,0.433976,0.047474,0.455606,0.155646,0.051901,-0.000682,-0.065924,1.0
1814,2006.0,1055.0,10.0,103528.0,NaN,32319.0,3591.0,10773.0,3591.0,NaN,...,9.659223,9.720913,0.433028,0.042454,0.452227,0.164993,0.055380,-0.000486,-0.061690,1.0
2532,2007.0,1055.0,14.0,103893.0,NaN,32319.0,3591.0,10773.0,3591.0,NaN,...,13.475403,13.566680,0.420867,0.044612,0.440520,0.170630,0.055449,0.002268,-0.091278,1.0
3262,2008.0,1055.0,20.0,104206.0,19.19275,32319.0,3591.0,10773.0,3591.0,NaN,...,19.192753,19.335815,0.415672,0.059746,0.442084,0.163610,0.054227,0.003123,-0.143062,1.0
4042,2009.0,1055.0,16.0,104239.0,NaN,32319.0,3591.0,10773.0,3591.0,NaN,...,15.349341,15.474187,0.385288,0.114573,0.435144,0.168930,0.060891,0.002331,-0.124846,1.0
4798,2010.0,1055.0,12.0,104430.0,NaN,NaN,NaN,NaN,NaN,208884.0,...,11.490951,11.588605,0.378493,0.109250,0.424915,0.187900,0.065447,-0.000357,-0.097654,1.0
5608,2011.0,1055.0,15.0,104303.0,NaN,NaN,NaN,NaN,NaN,208472.0,...,14.381178,14.390422,0.386028,0.100264,0.429046,0.205553,0.068537,0.001458,-0.009244,1.0
6447,2012.0,1055.0,10.0,104392.0,NaN,NaN,NaN,NaN,NaN,208470.0,...,9.579278,9.593707,0.389121,0.082477,0.424099,0.206341,0.069746,-0.001976,-0.014428,1.0
7281,2013.0,1055.0,21.0,103931.0,20.20571,NaN,NaN,NaN,NaN,207704.0,...,20.205713,20.221084,0.389063,0.076014,0.421070,0.191985,0.061453,-0.000010,-0.015370,1.0


In [5]:
stop stop

SyntaxError: invalid syntax (<ipython-input-5-a0f076e259f0>, line 1)

In [ ]:


deaths_pop['county_code'] = deaths_pop['county_code'].apply(lambda x: int(x))
deaths_pop['county_name'] = deaths_pop['county_code'].apply(lambda x: name_codes[x])
deaths_pop['state'] = deaths_pop['county_name'].apply(lambda x: state_extract(x))


state_dums = pd.get_dummies(deaths_pop['state'])
deaths_pop = pd.concat([deaths_pop,state_dums], axis = 1)


deaths_pop = floater(deaths_pop) #convert any remaining fields to float
deaths_pop['year'].apply(lambda x: str(x)) #need year to be in string for dummies
year_dums = pd.get_dummies(deaths_pop['year'])
deaths_pop = pd.concat([deaths_pop,year_dums], axis = 1) #not using these anymore in features

#merging naxolone access policies (Narcan)

deaths_pop = pd.merge(deaths_pop, naxo, how='left', left_on=['year', 'state'], right_on=['year', 'state'])


## Adding auto-regressive features

deaths_pop = deaths_pop.sort_values(by=['county_code', 'year'])
deaths_pop['prior_year_death_rate_100k'] = deaths_pop['drug_death_rate_100k'].shift(1)
deaths_pop['pre_prior_year_death_rate_100k'] = deaths_pop['drug_death_rate_100k'].shift(2)
deaths_pop['prior_year_death_growth_rate'] = (deaths_pop['prior_year_death_rate_100k'] - deaths_pop['pre_prior_year_death_rate_100k'])/deaths_pop['pre_prior_year_death_rate_100k']
deaths_pop['three_back_year_death_rate_100k'] = deaths_pop['drug_death_rate_100k'].shift(3)
deaths_pop['two_year_period_death_growth_rate'] = (deaths_pop['prior_year_death_rate_100k'] - deaths_pop['three_back_year_death_rate_100k'])/ deaths_pop['three_back_year_death_rate_100k']
deaths_pop['two_three_year_death_growth_rate'] = (deaths_pop['pre_prior_year_death_rate_100k']-deaths_pop['three_back_year_death_rate_100k'])/ deaths_pop['three_back_year_death_rate_100k']
deaths_pop['years_since_14'] = 2014 - deaths_pop['year']



deaths_pop.drop(886, inplace=1)
deaths_pop = deaths_pop.sort_index() #to undo sorting
death_df = deaths_pop  ## going to clean death_df to include only counties w/ auto-regressive features


death_df['count_county_code'] = np.nan
death_df.reset_index(inplace=1)
death_df = dict_applier(death_df)

deaths_pop = death_df[death_df['count_county_code']>=4] # 4 b/c 3 auto-regressive features


##### Implementing Model Below


median_drug_rate = np.median(deaths_pop['drug_death_rate_100k'])
avg_drug_rate = np.mean(deaths_pop['drug_death_rate_100k'])

y = deaths_pop['drug_death_rate_100k']

# v1 features = deaths_pop[['year','pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','pop_hisp_prop','population','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'constant', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']]
# v2 features, removed hisp, years, population below = deaths_pop[['pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','pop_hisp_prop','population','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'constant', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY', 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]]

features = deaths_pop[['prior_year_death_rate_100k', 'pre_prior_year_death_rate_100k', 'prior_year_death_growth_rate','three_back_year_death_rate_100k', 'two_year_period_death_growth_rate', 'two_three_year_death_growth_rate', 'national_fentanyl_seizures', 'naxo_crim', 'naxo_civ', 'naxo_third', 'pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'population_est', 'pop_change_rate','natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate', 'constant', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']]
#8/25 added autoregressive features

features.to_csv('/Users/HudsonCavanagh/Dropbox/Capstone/csv_output/full_features_cdc_states_v7.csv')

X = StandardScaler().fit_transform(features)
X = pd.DataFrame(X, columns=features.columns)

#will be using cross_val, this is to validate most successful model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)


#PCA for later plots & model testing

pca_2 = PCA(n_components=2) # for visualization
pca_3 = PCA(n_components=3)
pca_4 = PCA(n_components=4)
xPC_2 = pca_2.fit_transform(X) # for visualization
xPC_3 = pca_3.fit_transform(X)
xPC_4 = pca_4.fit_transform(X)

stratk = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=66) #for cross_val

dt_simple = DecisionTreeRegressor(min_samples_leaf=4, min_samples_split=6, max_depth=6)
dt_simple.fit(X,y)
s_dt = cross_val_score(dt_simple, X, y, cv=stratk, n_jobs=-1, scoring='roc_auc')

print("Decision Tree CV score", s_dt.mean())

rf = RandomForestRegressor()
rf_cv = RandomForestRegressor()

rf.fit(X_train, y_train)
ypred_rf = rf.predict(X_test)
s_rf_cv = cross_val_score(rf_cv, X, y, cv=stratk, n_jobs=-1, scoring='roc_auc')

print("Random Forest CV score", s_rf_cv.mean())

# code for GS RF model

# gs_rf = GridSearchCV(rf, {'min_samples_split': split_vals, 'max_depth': depths, 'max_features': max_feats}, cv=15)
# s_gs_rf_cv = cross_val_score(gs_rf, X, y, cv=stratk, n_jobs=-1)
# s_gs_rf_cv.mean()


#Bagging/ Bootstrapped Model
bc_cv = BaggingRegressor(n_estimators=100)
# bc_cv.fit(X_train,y_train)
s_bc = cross_val_score(bc_cv, X, y, cv=stratk, n_jobs=-1, scoring='roc_auc')

print("{} Score:\t{:0.3} ± {:0.3}".format("Bagging Trees", s_bc.mean(), s_bc.std()))

# ## Looking at Feature Importance
rel_feature_import = sorted(zip(rf.feature_importances_, features.columns), reverse=True)
rel_feature_import = pd.DataFrame(rel_feature_import)
print("RF feature importance:", rel_feature_import) #pop was .74 before outliar removal


# ## Implementing Kaggle-Darling XG Boost

xg_model = xgboost.XGBRegressor()
xg_model_cv = xgboost.XGBRegressor()
xg_model.fit(X_train, y_train)
s_xg_cv = cross_val_score(xg_model_cv, X, y, cv=stratk, n_jobs=-1, scoring='roc_auc')

# make predictions for test data
y_pred_xg = xg_model.predict(X_test)
print("XG Boost cross_val", s_xg_cv.mean())


# ### Implementing GradientBoosting Regressor to see if this is diff from XG Boost

gb_tree = GradientBoostingRegressor(n_estimators=1000)
gb_tree_cv = GradientBoostingRegressor(n_estimators=1000, max_depth=10)

gb_tree.fit(X_train, y_train)
ypred_gbtree = gb_tree.predict(X_test)


s_gb_tree_cv = cross_val_score(gb_tree_cv, X, y, cv=stratk, n_jobs=-1, scoring='roc_auc')

print("Gradient Boosted cross_val", s_gb_tree_cv.mean())


###### Looking at the biggest difference between predicted and observed values - commented out
# ypred_gbtree = pd.DataFrame(ypred_gbtree)
# y_test = pd.DataFrame(y_test)
# pred_act_diff = y_test.merge(deaths_pop, how='left', left_index=1, right_index=1)
# pred_act_diff['death_rate_actual_ytest'] = pred_act_diff['drug_death_rate_100k_x']
# pred_act_diff = pred_act_diff.merge(ypred_gbtree, how='left', left_index=1, right_index=1)
# pred_act_diff['death_rate_pred_gb'] = pred_act_diff[0]
# pred_act_diff = pred_act_diff[pred_act_diff['death_rate_pred_gb'].isnull()==False]
# pred_act_diff['residual'] = pred_act_diff['death_rate_pred_gb'] - pred_act_diff['death_rate_actual_ytest']
#
# #note positive means that actual > predicted
# pred_act_diff[['residual', 'death_rate_pred_gb', 'death_rate_actual_ytest', 'county_name', 'year']]
#
# predict_eval = pred_act_diff[['residual', 'death_rate_pred_gb', 'death_rate_actual_ytest', 'county_name', 'year']].sort('residual')
# # predict_eval.head(5)
# # predict_eval.tail(5)



## Now going to use Classification models - Random Forest and XG Boost

y_bin = y.apply(lambda x: 1 if x > median_drug_rate else 0) # new target for classification
rf_c_cv = RandomForestClassifier()
s_rf_cv = cross_val_score(rf_c_cv, X, y_bin, cv=stratk, n_jobs=-1)
print("random forest, median_drug_rate classifier AUC", s_rf_cv.mean())

xg_class_model_cv = xgboost.XGBClassifier()
s_xg_cv = cross_val_score(xg_class_model_cv, X, y_bin, cv=stratk, n_jobs=-1)
print("XG Boost, median drug rate classifier AUC", s_xg_cv.mean())


## Predicting change in 2014, using 2003-2013 data (hardest model to get accurate)

input_2013 = deaths_pop[deaths_pop['year']<2014]
input_2013.reset_index(inplace=1) # added this 8/23 to alleviate nans below
test_2014 = deaths_pop[deaths_pop['year']==2014]
test_2014.reset_index(inplace=1) # added this 8/23 to alleviate nans below

#creating regression training
y_03_13 = input_2013['drug_death_rate_100k']

#regression target
y_14 = test_2014['drug_death_rate_100k']

#creating classification (bigger/ smaller) train & target
y_delta_bin_03_13 = (input_2013['drug_death_rate_100k']-input_2013['prior_year_death_rate_100k']).apply(lambda x: 1 if x >= 0 else 0) #one means increase
y_delta_bin_14 = (test_2014['drug_death_rate_100k']-test_2014['prior_year_death_rate_100k']).apply(lambda x: 1 if x >= 0 else 0) #one means increase

#creating regression train & target for drug rate growth
y_delta_grow_03_13 = ((input_2013['drug_death_rate_100k']-input_2013['prior_year_death_rate_100k'])/input_2013['prior_year_death_rate_100k'])
y_delta_grow_14 = ((test_2014['drug_death_rate_100k']-test_2014['prior_year_death_rate_100k'])/test_2014['prior_year_death_rate_100k'])


#8/22 added prior year death rate as feature
features_03_13 = input_2013[['prior_year_death_rate_100k','pre_prior_year_death_rate_100k', 'prior_year_death_growth_rate','three_back_year_death_rate_100k', 'two_year_period_death_growth_rate', 'two_three_year_death_growth_rate', 'national_fentanyl_seizures','naxo_crim', 'naxo_civ', 'naxo_third', 'pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'population_est', 'pop_change_rate','natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate', 'constant', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']]
features_14 = test_2014[['prior_year_death_rate_100k','pre_prior_year_death_rate_100k', 'prior_year_death_growth_rate','three_back_year_death_rate_100k', 'two_year_period_death_growth_rate', 'two_three_year_death_growth_rate','national_fentanyl_seizures','naxo_crim', 'naxo_civ', 'naxo_third', 'pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'population_est', 'pop_change_rate','natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate', 'constant', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']]
X_03_13 = StandardScaler().fit_transform(features_03_13)
X_03_13 = pd.DataFrame(X_03_13, columns=features_03_13.columns)

X_14 = StandardScaler().fit_transform(features_14)
X_14 = pd.DataFrame(X_14, columns=features_14.columns)

gb_14 = GradientBoostingRegressor(n_estimators=1000, max_depth=10)
gb_14.fit(X_03_13, y_03_13)
ypred_14 = gb_14.predict(X_14)

print("GBoosted rsq score, overall drug death rate regression", r2_score(y_14, ypred_14))

#.6075 before fent feature


#logistic regression to test direction of  features

logreg_basic = LogisticRegression(solver='liblinear')
logreg_basic.fit(X_03_13, y_delta_bin_03_13)
y_bin_preds_log = logreg_basic.predict(X_14)



# X_03_13_ = pd.merge(X_03_13_mod, state_features_train, how='left', left_index=1, right_index=1) -- pretty sure to be removed


# Grid Search implementation on LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

logreg = LogisticRegression(solver='liblinear')
C_vals = [0.01, 0.1, .25, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals}, verbose=False, cv=15)
gs.fit(X_03_13, y_delta_bin_03_13)
y_bin_preds_gs_logis = gs.predict(X_14)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

lr_basic_cm = confusion_matrix(y_delta_bin_14, y_bin_preds_gs_logis)
lr_basic_cm_df = pd.DataFrame(lr_basic_cm)
print(lr_basic_cm_df)

coefs_variables = pd.DataFrame(zip(gs.coef_[0], X_03_13.columns))
coefs_variables.columns = ['coef', 'feature']
coefs_variables['abs_coef'] = abs(coefs_variables['coef'])
coefs_variables = coefs_variables.sort_values(by='abs_coef', ascending=0)
print("coefficients of logistic regressions:", coefs_variables)


# ## How accurately can we classify communities as compared to the median rate?

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectFromModel

y_bin_03_13 = y_03_13.apply(lambda x: 1 if x > median_drug_rate else 0)
y_bin_14_obs = y_14.apply(lambda x: 1 if x > median_drug_rate else 0)


xg_class_model_14 = xgboost.XGBClassifier()
xg_class_model_14.fit(X_03_13, y_bin_03_13)
y_class_14 = xg_class_model_14.predict(X_14)

xg_14_cm = confusion_matrix(y_bin_14_obs, y_class_14, labels=xg_class_model_14.classes_)
# xg_14_cm = pd.DataFrame(xg_14_cm, columns=xg_class_model_14.classes_, index=xg_class_model_14.classes_)
feat_import_XG_class_14 = pd.Series(zip(X_03_13, xg_class_model_14.feature_importances_))
print(feat_import_XG_class_14)

#Using tree-based feature importance to reduce my features -- predicting against median

xg_class_model_14_select = SelectFromModel(xg_class_model_14, prefit=True)
X_03_13_new = xg_class_model_14_select.transform(X_03_13)
X_14_new = xg_class_model_14_select.transform(X_14)


xg_class_model_14_mod = xgboost.XGBClassifier()
xg_class_model_14_mod.fit(X_03_13_new, y_bin_03_13)
y_class_xg_14_mod = xg_class_model_14_mod.predict(X_14_new)
xg_14_mod_cm = confusion_matrix(y_class_xg_14_mod, y_bin_14_obs, labels=xg_class_model_14.classes_)


print('f1:', sklearn.metrics.f1_score(y_bin_14_obs,y_class_xg_14_mod),
'accuracy:', accuracy_score(y_bin_14_obs,y_class_xg_14_mod))


# plot_confusion_matrix(xg_14_mod_cm)


print('f1:', sklearn.metrics.f1_score(y_bin_14_obs,y_class_14),
'accuracy:', accuracy_score(y_bin_14_obs,y_class_14))


# ### How accurately can we predict the directional movement of communities in 2014, e.g. can our model predict which communties were going to get worse in 2014?

xg_delta_model_14 = xgboost.XGBClassifier()
xg_delta_model_14.fit(X_03_13, y_delta_bin_03_13)
y_delta_14_pred = xg_delta_model_14.predict(X_14)

xg_delta_14_cm = confusion_matrix(y_delta_bin_14, y_delta_14_pred)
# xg_14_cm = pd.DataFrame(xg_14_cm, columns=xg_class_model_14.classes_, index=xg_class_model_14.classes_)
plot_confusion_matrix(xg_delta_14_cm)


print('f1:', sklearn.metrics.f1_score(y_delta_bin_14, y_delta_14_pred),
'accuracy:', accuracy_score(y_delta_bin_14, y_delta_14_pred))



xg_delta_model_14_select = SelectFromModel(xg_delta_model_14, prefit=True)
X_03_13_new_delta = xg_delta_model_14_select.transform(X_03_13)
X_14_new_delta = xg_delta_model_14_select.transform(X_14)


xg_delta_model_14_mod = xgboost.XGBClassifier()
xg_delta_model_14_mod.fit(X_03_13_new_delta, y_delta_bin_03_13)
y_delta_xg_14_mod = xg_delta_model_14_mod.predict(X_14_new_delta)
xg_14_mod_cm = confusion_matrix(y_delta_bin_14, y_class_xg_14_mod, labels=xg_class_model_14.classes_)

print("XG Boost classification model auc score ('14 delta'):", sklearn.metrics.roc_auc_score(y_delta_bin_14,y_class_xg_14_mod),
      "naive auc score:", sklearn.metrics.roc_auc_score(y_delta_bin_14, naive_guesses))

# xg_bin_delta_14_cm = confusion_matrix(y_delta_bin_14, naive_guesses)
# plot_confusion_matrix(xg_bin_delta_14_cm)


gb_delta_14 = GradientBoostingRegressor(n_estimators=1000)

gb_delta_14.fit(X_03_13, y_delta_grow_03_13)
gb_delta_14_ypred = gb_delta_14.predict(X_14)
r2_score(y_delta_grow_14, gb_delta_14_ypred)

print(y_delta_grow_14.mean(), gb_delta_14_ypred.mean())


# In[108]:

gb_delta_model_14_select = SelectFromModel(gb_delta_14, prefit=True)
X_03_13_new_delta_gb = gb_delta_model_14_select.transform(X_03_13)
X_14_new_delta_gb = gb_delta_model_14_select.transform(X_14)

#lets try this with both the features from X_03_13_new_delta_gb and X_03_13_new_delta (same for test)


gb_delta_model_14_mod = xgboost.XGBClassifier()
gb_delta_model_14_mod.fit(X_03_13_new_delta_gb, y_delta_bin_03_13)
y_delta_gb_14_mod = gb_delta_model_14_mod.predict(X_14_new_delta_gb)
# gb_14_mod_cm = confusion_matrix(y_delta_grow_14, y_delta_gb_14_mod, labels=xg_class_model_14.classes_)

r2_score(y_delta_grow_14, y_delta_gb_14_mod)


# In[109]:

gb_delta_model_14_xg_feats = xgboost.XGBClassifier()
gb_delta_model_14_xg_feats.fit(X_03_13_new_delta, y_delta_bin_03_13)
y_delta_gb_14_xg_feats = gb_delta_model_14_xg_feats.predict(X_14_new_delta)
# gb_14_mod_cm = confusion_matrix(y_delta_grow_14, y_delta_gb_14_mod, labels=xg_class_model_14.classes_)

r2_score(y_delta_grow_14, y_delta_gb_14_xg_feats)


# In[110]:

sklearn.metrics.f1_score(y_delta_gb_14_xg_feats, y_delta_bin_14)


print('f1:', sklearn.metrics.f1_score(y_delta_bin_14, y_delta_gb_14_mod),
'accuracy:', accuracy_score(y_delta_bin_14, y_delta_gb_14_mod))


### Constructing Final Batch of Features

features_03_13_mod = input_2013[['prior_year_death_rate_100k','pre_prior_year_death_rate_100k', 'prior_year_death_growth_rate','three_back_year_death_rate_100k', 'two_year_period_death_growth_rate', 'two_three_year_death_growth_rate','pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'population_est', 'pop_change_rate','natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate', 'constant']]

state_features_train = input_2013[['AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']]
state_features_test = test_2014[['AK', 'AL', 'AR', 'AZ', 'CA', 'CO','CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME','MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'NE', 'NH','NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI','SC','SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']]


features_14_mod = test_2014[['prior_year_death_rate_100k','pre_prior_year_death_rate_100k', 'prior_year_death_growth_rate','three_back_year_death_rate_100k', 'two_year_period_death_growth_rate', 'two_three_year_death_growth_rate', 'pop_sub_15_prop','pop_15-34_prop','pop_35-54_prop','pop_55+_prop','pop_asian_prop', 'pop_white_prop','pop_black_prop','med_hh_income', 'unemp_rate', 'perc_pop_working', 'pov_rate', 'pov_rate_youth', 'population_est', 'pop_change_rate','natural_pop_growth_rate', 'intl_migrate_rate', 'dom_migrate_rate', 'constant']]

X_03_13_mod = StandardScaler().fit_transform(features_03_13_mod)
X_03_13_mod = pd.DataFrame(X_03_13_mod, columns=features_03_13_mod.columns)

# print(X_03_13_mod.isnull().sum().sum()) -- none

X_14_mod = StandardScaler().fit_transform(features_14_mod)
X_14_mod = pd.DataFrame(X_14_mod, columns=features_14_mod.columns)

X_03_13_mod = pd.merge(X_03_13_mod, state_features_train, how='left', left_index=1, right_index=1)
# print(X_03_13_mod.isnull().sum().sum()) -- none
X_14_mod = pd.merge(X_14_mod, state_features_test, how='left', left_index=1, right_index=1)

gb_delta_14_mod = GradientBoostingRegressor(n_estimators=10000, learning_rate=.02)
gb_delta_14_mod.fit(X_03_13_mod, y_delta_grow_03_13) # works
gb_delta_14_ypred_mod = gb_delta_14.predict(X_14_mod)

print("2014 delta regression, GBoost r2 score:", r2_score(gb_delta_14_ypred_mod, y_delta_grow_14))

# gb_delta_14_ypred_mod = pd.DataFrame(gb_delta_14_ypred_mod)
# worst_off_2014 = pd.merge(test_2014, gb_delta_14_ypred, how='left', left_index=1, right_index=1)



## Grid Search Optimizing Gradient Boosted Trees -- commented out because takes too long
#
# max_feature_list = ['auto', 'sqrt']
# param_test = {'max_depth':range(3,15,2), 'max_features':max_feature_list, 'min_samples_split':range(3,9), 'min_samples_leaf':range(2,8)}
#
# gb_delta_14_reg = GradientBoostingRegressor(n_estimators=1000, learning_rate=.05)
# gb_delta_14_class = GradientBoostingClassifier(n_estimators=1000, learning_rate=.05)
#
# gs_delta_14_reg = GridSearchCV(gb_delta_14_reg, param_test, verbose=False, cv=15)
# gs_delta_14_reg.fit(X_03_13_mod, y_delta_grow_03_13)
# y_delta_14_gs_preds = gs_delta_14_reg.predict(X_14_mod)
#
# print("2014 delta regression, GBoost r2 score:", r2_score(gb_delta_14_ypred_mod, y_delta_grow_14)
#
# print(gs_delta_14_reg.best_params_)
